# Re-Imaging Price Trends - Comparative Model Training

**Purpose**: Train models on both original and filled data for performance comparison

**Prerequisites**: `1_image_generation.ipynb` completed for both data versions

In [ ]:
# Environment setup
!pip install -r requirements.txt

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ReImaging_Price_Trends')
print(f"Current directory: {os.getcwd()}")

# GPU check
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# GPU memory monitoring
def check_gpu_memory():
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        total_memory = torch.cuda.get_device_properties(device).total_memory
        allocated = torch.cuda.memory_allocated(device)
        print(f"GPU memory: {allocated/1024**3:.1f}/{total_memory/1024**3:.1f}GB ({allocated/total_memory*100:.1f}%)")
        return allocated, total_memory
    else:
        print("GPU unavailable")
        return 0, 0

def cleanup_memory():
    import gc
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()
    print("Memory cleaned")

check_gpu_memory()

In [ ]:
# =============================================================================
# COMPARATIVE TRAINING CONFIGURATION
# =============================================================================

# Training method
USE_ENSEMBLE = True    # True: ensemble (paper method), False: single model
ENSEMBLE_RUNS = 5      # Number of ensemble models

print(f"=== COMPARATIVE TRAINING SETUP ===")
print(f"Training method: {'Ensemble (' + str(ENSEMBLE_RUNS) + ' models)' if USE_ENSEMBLE else 'Single model'}")
print(f"Data versions: Both original AND filled")
print(f"Purpose: Performance comparison between datasets")
print(f"")
print(f"Output models:")
print(f"  - CNN5d (original data): CNN5d_I5R5.tar")
print(f"  - CNN5d (filled data):   CNN5d_I5R5_filled.tar")
print(f"  - Same pattern for CNN20d and CNN60d")
print(f"")
print(f"This allows direct performance comparison!")

In [ ]:
# Check both original and filled data availability
import os

def check_data_version(version):
    data_dir = 'img_data_reconstructed' if version == 'original' else 'img_data_reconstructed_filled'
    
    required_dirs = ['weekly_5d', 'monthly_20d', 'quarterly_60d']
    all_ready = True
    
    print(f"\n{version.upper()} DATA:")
    for dir_name in required_dirs:
        dir_path = os.path.join(data_dir, dir_name)
        if os.path.exists(dir_path):
            dat_files = len([f for f in os.listdir(dir_path) if f.endswith('.dat')])
            feather_files = len([f for f in os.listdir(dir_path) if f.endswith('.feather')])
            print(f"  ✓ {dir_name}: {dat_files} .dat, {feather_files} .feather files")
        else:
            print(f"  ✗ {dir_name}: MISSING")
            all_ready = False
    
    return all_ready

# Check both versions
original_ready = check_data_version('original')
filled_ready = check_data_version('filled')

print(f"\n=== DATA AVAILABILITY SUMMARY ===")
print(f"Original data: {'✅ Ready' if original_ready else '❌ Missing'}")
print(f"Filled data:   {'✅ Ready' if filled_ready else '❌ Missing'}")

if original_ready and filled_ready:
    print(f"\n🎉 Both datasets ready for comparative training!")
elif original_ready:
    print(f"\n⚠️  Only original data available. Run filled data generation first.")
elif filled_ready:
    print(f"\n⚠️  Only filled data available. Run original data generation first.")
else:
    print(f"\n❌ No data available. Run 1_image_generation.ipynb for both versions.")

# Set training flags
CAN_TRAIN_ORIGINAL = original_ready
CAN_TRAIN_FILLED = filled_ready

# CNN5d Training (Original vs Filled Data)

In [ ]:
# CNN5d Training - Original Data
print(f"=== CNN5d Training - ORIGINAL Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN5d_I5R5{'_run1-5' if USE_ENSEMBLE else ''}.tar")

if CAN_TRAIN_ORIGINAL:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on original data...")
        !python train.py --model CNN5d --image_days 5 --pred_days 5 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version original
    else:
        print(f"\nTraining single model on original data...")
        !python train.py --model CNN5d --image_days 5 --pred_days 5 --use_original_format --data_version original
    
    print("✅ CNN5d (original) training completed")
else:
    print("❌ Original data not available - skipping")

cleanup_memory()

In [ ]:
# CNN5d Training - Filled Data
print(f"=== CNN5d Training - FILLED Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN5d_I5R5_filled{'_run1-5' if USE_ENSEMBLE else ''}.tar")

if CAN_TRAIN_FILLED:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on filled data...")
        !python train.py --model CNN5d --image_days 5 --pred_days 5 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version filled
    else:
        print(f"\nTraining single model on filled data...")
        !python train.py --model CNN5d --image_days 5 --pred_days 5 --use_original_format --data_version filled
    
    print("✅ CNN5d (filled) training completed")
else:
    print("❌ Filled data not available - skipping")

cleanup_memory()

# CNN20d Training (Original vs Filled Data)

In [ ]:
# CNN20d Training - Original Data
print(f"=== CNN20d Training - ORIGINAL Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN20d_I20R20{'_run1-5' if USE_ENSEMBLE else ''}.tar")

if CAN_TRAIN_ORIGINAL:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on original data...")
        !python train.py --model CNN20d --image_days 20 --pred_days 20 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version original
    else:
        print(f"\nTraining single model on original data...")
        !python train.py --model CNN20d --image_days 20 --pred_days 20 --use_original_format --data_version original
    
    print("✅ CNN20d (original) training completed")
else:
    print("❌ Original data not available - skipping")

cleanup_memory()

In [ ]:
# CNN20d Training - Filled Data
print(f"=== CNN20d Training - FILLED Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN20d_I20R20_filled{'_run1-5' if USE_ENSEMBLE else ''}.tar")

if CAN_TRAIN_FILLED:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on filled data...")
        !python train.py --model CNN20d --image_days 20 --pred_days 20 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version filled
    else:
        print(f"\nTraining single model on filled data...")
        !python train.py --model CNN20d --image_days 20 --pred_days 20 --use_original_format --data_version filled
    
    print("✅ CNN20d (filled) training completed")
else:
    print("❌ Filled data not available - skipping")

cleanup_memory()

# CNN60d Training (Original vs Filled Data)

In [ ]:
# CNN60d Training - Original Data
print(f"=== CNN60d Training - ORIGINAL Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN60d_I60R60{'_run1-5' if USE_ENSEMBLE else ''}.tar")
print(f"Warning: Most memory intensive!")

if CAN_TRAIN_ORIGINAL:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on original data...")
        !python train.py --model CNN60d --image_days 60 --pred_days 60 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version original --batch_size 64
    else:
        print(f"\nTraining single model on original data...")
        !python train.py --model CNN60d --image_days 60 --pred_days 60 --use_original_format --data_version original --batch_size 64
    
    print("✅ CNN60d (original) training completed")
else:
    print("❌ Original data not available - skipping")

cleanup_memory()

In [ ]:
# CNN60d Training - Filled Data
print(f"=== CNN60d Training - FILLED Data ===")
print(f"Method: {'Ensemble' if USE_ENSEMBLE else 'Single model'}")
print(f"Output: CNN60d_I60R60_filled{'_run1-5' if USE_ENSEMBLE else ''}.tar")
print(f"Warning: Most memory intensive!")

if CAN_TRAIN_FILLED:
    check_gpu_memory()
    
    if USE_ENSEMBLE:
        print(f"\nTraining ensemble ({ENSEMBLE_RUNS} models) on filled data...")
        !python train.py --model CNN60d --image_days 60 --pred_days 60 --ensemble --ensemble_runs {ENSEMBLE_RUNS} --use_original_format --data_version filled --batch_size 64
    else:
        print(f"\nTraining single model on filled data...")
        !python train.py --model CNN60d --image_days 60 --pred_days 60 --use_original_format --data_version filled --batch_size 64
    
    print("✅ CNN60d (filled) training completed")
else:
    print("❌ Filled data not available - skipping")

cleanup_memory()

# Comparative Portfolio Evaluation

In [ ]:
# CNN5d Comparative Evaluation
print(f"=== CNN5d COMPARATIVE EVALUATION ===")
print(f"Strategy: Weekly (I5/R5)")
print(f"Paper benchmark: Sharpe = 7.15")
print(f"")

ensemble_flag = "--ensemble" if USE_ENSEMBLE else ""

if CAN_TRAIN_ORIGINAL:
    print("📊 CNN5d - Original Data:")
    !python test.py --model CNN5d --image_days 5 --pred_days 5 {ensemble_flag} --use_original_format --data_version original
    cleanup_memory()
else:
    print("❌ CNN5d original model not available")

print("\n" + "="*50)

if CAN_TRAIN_FILLED:
    print("📊 CNN5d - Filled Data:")
    !python test.py --model CNN5d --image_days 5 --pred_days 5 {ensemble_flag} --use_original_format --data_version filled
    cleanup_memory()
else:
    print("❌ CNN5d filled model not available")

print("\n✅ CNN5d comparative evaluation completed")

In [ ]:
# CNN20d Comparative Evaluation
print(f"=== CNN20d COMPARATIVE EVALUATION ===")
print(f"Strategy: Monthly (I20/R20)")
print(f"Paper benchmark: Sharpe = 2.16")
print(f"")

ensemble_flag = "--ensemble" if USE_ENSEMBLE else ""

if CAN_TRAIN_ORIGINAL:
    print("📊 CNN20d - Original Data:")
    !python test.py --model CNN20d --image_days 20 --pred_days 20 {ensemble_flag} --use_original_format --data_version original
    cleanup_memory()
else:
    print("❌ CNN20d original model not available")

print("\n" + "="*50)

if CAN_TRAIN_FILLED:
    print("📊 CNN20d - Filled Data:")
    !python test.py --model CNN20d --image_days 20 --pred_days 20 {ensemble_flag} --use_original_format --data_version filled
    cleanup_memory()
else:
    print("❌ CNN20d filled model not available")

print("\n✅ CNN20d comparative evaluation completed")

In [ ]:
# CNN60d Comparative Evaluation
print(f"=== CNN60d COMPARATIVE EVALUATION ===")
print(f"Strategy: Quarterly (I60/R60)")
print(f"Paper benchmark: Sharpe = 0.37")
print(f"")

ensemble_flag = "--ensemble" if USE_ENSEMBLE else ""

if CAN_TRAIN_ORIGINAL:
    print("📊 CNN60d - Original Data:")
    !python test.py --model CNN60d --image_days 60 --pred_days 60 {ensemble_flag} --use_original_format --data_version original
    cleanup_memory()
else:
    print("❌ CNN60d original model not available")

print("\n" + "="*50)

if CAN_TRAIN_FILLED:
    print("📊 CNN60d - Filled Data:")
    !python test.py --model CNN60d --image_days 60 --pred_days 60 {ensemble_flag} --use_original_format --data_version filled
    cleanup_memory()
else:
    print("❌ CNN60d filled model not available")

print("\n✅ CNN60d comparative evaluation completed")

# Comparative Results Analysis

In [ ]:
# Comparative Results Analysis
import json
import pandas as pd

print("=" * 80)
print("COMPARATIVE RESULTS ANALYSIS")
print("=" * 80)

def load_result(model, data_version, ensemble=USE_ENSEMBLE):
    """Load performance result from JSON file"""
    eval_type = "ensemble" if ensemble else "single"
    version_suffix = "_filled" if data_version == 'filled' else ""
    
    result_file = f"results/{model}_I{model[3:]}R{model[3:]}{version_suffix}_{eval_type}_performance.json"
    
    try:
        with open(result_file, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return None

# Create comparison table
comparison_data = []
models = ['CNN5d', 'CNN20d', 'CNN60d']
strategies = ['Weekly (I5/R5)', 'Monthly (I20/R20)', 'Quarterly (I60/R60)']
paper_benchmarks = [7.15, 2.16, 0.37]

for i, (model, strategy, paper_sharpe) in enumerate(zip(models, strategies, paper_benchmarks)):
    print(f"\n{'='*60}")
    print(f"{model} - {strategy}")
    print(f"Paper benchmark: {paper_sharpe}")
    print(f"{'='*60}")
    
    # Load results
    original_result = load_result(model, 'original')
    filled_result = load_result(model, 'filled')
    
    # Extract key metrics
    if original_result:
        orig_sharpe = original_result['ls_sharpe_ratio']
        orig_return = original_result['ls_annual_return'] * 100
        orig_turnover = original_result['monthly_turnover'] * 100
        print(f"\n📊 ORIGINAL Data:")
        print(f"   Long-Short Sharpe: {orig_sharpe:.2f}")
        print(f"   Annual Return: {orig_return:.2f}%")
        print(f"   Monthly Turnover: {orig_turnover:.1f}%")
    else:
        orig_sharpe = orig_return = orig_turnover = None
        print(f"\n❌ ORIGINAL Data: No results")
    
    if filled_result:
        fill_sharpe = filled_result['ls_sharpe_ratio']
        fill_return = filled_result['ls_annual_return'] * 100
        fill_turnover = filled_result['monthly_turnover'] * 100
        print(f"\n📊 FILLED Data:")
        print(f"   Long-Short Sharpe: {fill_sharpe:.2f}")
        print(f"   Annual Return: {fill_return:.2f}%")
        print(f"   Monthly Turnover: {fill_turnover:.1f}%")
    else:
        fill_sharpe = fill_return = fill_turnover = None
        print(f"\n❌ FILLED Data: No results")
    
    # Performance comparison
    if orig_sharpe is not None and fill_sharpe is not None:
        sharpe_diff = fill_sharpe - orig_sharpe
        return_diff = fill_return - orig_return
        
        print(f"\n🔍 COMPARISON:")
        print(f"   Sharpe improvement: {sharpe_diff:+.2f} ({sharpe_diff/orig_sharpe*100:+.1f}%)")
        print(f"   Return improvement: {return_diff:+.2f}pp")
        
        if fill_sharpe > orig_sharpe:
            print(f"   🎉 Filled data WINS by {sharpe_diff:.2f} Sharpe points!")
        elif orig_sharpe > fill_sharpe:
            print(f"   🏆 Original data WINS by {-sharpe_diff:.2f} Sharpe points!")
        else:
            print(f"   🤝 TIE - Similar performance")
    
    # Store for summary table
    comparison_data.append({
        'Model': model,
        'Strategy': strategy,
        'Paper_Sharpe': paper_sharpe,
        'Original_Sharpe': orig_sharpe,
        'Filled_Sharpe': fill_sharpe,
        'Original_Return': orig_return,
        'Filled_Return': fill_return
    })

# Summary table
print(f"\n\n{'='*80}")
print(f"SUMMARY TABLE")
print(f"{'='*80}")

df = pd.DataFrame(comparison_data)
print(f"{'Model':<8} {'Strategy':<15} {'Paper':<8} {'Original':<10} {'Filled':<10} {'Winner':<10}")
print(f"{'-'*75}")

for _, row in df.iterrows():
    model = row['Model']
    strategy = row['Strategy'].split()[0]  # Weekly/Monthly/Quarterly
    paper = f"{row['Paper_Sharpe']:.2f}"
    orig = f"{row['Original_Sharpe']:.2f}" if row['Original_Sharpe'] is not None else "N/A"
    filled = f"{row['Filled_Sharpe']:.2f}" if row['Filled_Sharpe'] is not None else "N/A"
    
    # Determine winner
    if row['Original_Sharpe'] is not None and row['Filled_Sharpe'] is not None:
        if row['Filled_Sharpe'] > row['Original_Sharpe']:
            winner = "Filled"
        elif row['Original_Sharpe'] > row['Filled_Sharpe']:
            winner = "Original"
        else:
            winner = "Tie"
    else:
        winner = "N/A"
    
    print(f"{model:<8} {strategy:<15} {paper:<8} {orig:<10} {filled:<10} {winner:<10}")

print(f"\n💡 KEY INSIGHTS:")
if len([x for x in comparison_data if x['Original_Sharpe'] is not None and x['Filled_Sharpe'] is not None]) > 0:
    filled_wins = sum(1 for x in comparison_data if x['Original_Sharpe'] is not None and x['Filled_Sharpe'] is not None and x['Filled_Sharpe'] > x['Original_Sharpe'])
    total_comparisons = len([x for x in comparison_data if x['Original_Sharpe'] is not None and x['Filled_Sharpe'] is not None])
    
    print(f"   Filled data wins: {filled_wins}/{total_comparisons} models")
    print(f"   Missing data impact: {'Positive' if filled_wins > total_comparisons/2 else 'Negative' if filled_wins < total_comparisons/2 else 'Neutral'}")
    
    if filled_wins > total_comparisons/2:
        print(f"   🎯 Recommendation: Use FILLED data for better performance")
    elif filled_wins < total_comparisons/2:
        print(f"   🎯 Recommendation: Use ORIGINAL data for better performance")
    else:
        print(f"   🎯 Recommendation: Both datasets perform similarly")
else:
    print(f"   No comparative results available yet")

print(f"\n📁 Result files saved in: results/")
if os.path.exists('results'):
    result_files = [f for f in os.listdir('results') if f.endswith('.json')]
    for f in sorted(result_files):
        print(f"   📊 {f}")
